In [55]:
pip install optuna

Note: you may need to restart the kernel to use updated packages.


---

**Proje Tanımı:**
2020 yılının başından 2023 Eylül ayına kadar olan dönemi kapsayan bir veri seti bulunmaktadır. Bu veri seti, iş yerlerinin ay bazında gerçekleştirdiği işlem sayılarını içermektedir. Test dosyası oluşturulmamıştır ve katılımcılar, train veri seti içerisinde istedikleri şekilde ve formatta test verisini ayırabilirler. Projenin amacı, 2023 son çeyrek (Ekim - Kasım - Aralık) ayları için iş yerleri bazında işlem sayılarını (net_payment_count) tahmin etmek ve bu tahminleri sunmaktır.

**Veri Setleri:**

- `merchant_id`: Maskelenmiş iş yeri ID'si
- `month_id`: İşlemin yapıldığı ay (YYYYMM formatında)
- `merchant_source`: İş yerinin iyzico’ya katıldığı kaynak
- `settlement_period`: İş yerinin hak edişini alış sıklığı
- `working_type`: İş yerinin tipini gösterir
- `mcc_id`: İş yerinin satış yaptığı kategori bilgisini gösterir
- `merchant_segment`: İş yerinin iyzico içerisinde bulunduğu segmenti gösterir
- `net_payment_count`: İş yerinin ilgili ay içerisinde geçirdiği net (ödeme - iptal - iade) işlem sayısıdır

**Submission Dosyası:**

- `id`: Tahminlenmesi beklenen ay ve maskelenmiş iş yeri ID'si (Örn: 202312merchant_36004)
- `net_payment_count`: İş yerinin ilgili ayda gerçekleştirdiği işlem sayısı

**Dış Veri Kullanımı:**

Modelin kurulumunda faydalı olabilecek dönemsel enflasyon verisi, maaş ödenme günleri (SGK ve devlet memurlarının maaş günleri vb.), resmi ve dini bayramlara ilişkin veriler, döviz kurları, mevsime göre ülkemizde görülen ortalama sıcaklık verileri gibi her türlü dış veri kullanılabilir.

--- 


In [56]:
import warnings 
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,StandardScaler
pd.set_option('display.max_rows', 250)
pd.set_option('display.max_columns', 50)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
class path :
    train='/kaggle/input/iyzico-datathon/train.csv'
    submission='/kaggle/input/iyzico-datathon/sample_submission.csv'
    test='/kaggle/input/iyizicotest/test.csv'
# Modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay,roc_curve,roc_auc_score,classification_report
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz

import optuna
from catboost import CatBoostRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import TimeSeriesSplit,cross_val_score,KFold,TimeSeriesSplit

def custom_df(df):
    df.month_id=df.month_id.astype(str)
    df['date']=pd.to_datetime(df.month_id.str[:4]+'-'+df.month_id.str[-2:])
    df.merchant_id=list(map(lambda x :int(x[1]),df.merchant_id.str.split('_')))
    df['merchant_source_name_id']=list(map(lambda x :int(x[1]),df.merchant_source_name.str.split('-')))
    df['settlement_period_id']=list(map(lambda x :int(x[1]),df.settlement_period.str.split('-')))
    df['working_type_id']=list(map(lambda x :int(x[1]),df.working_type.str.split('-')))
    df['mcc_id']=list(map(lambda x :int(x[1]),df.mcc_id.str.split('_')))
    df['merchant_segment_id']=list(map(lambda x :int(x[1]),df.merchant_segment.str.split('-')))
    df.drop(['merchant_source_name','settlement_period','working_type','merchant_segment','month_id'],axis=1,inplace=True)
    return df

In [57]:
train=pd.read_csv(path.train)
submission=pd.read_csv(path.submission)
test=pd.read_csv(path.test)
submission['merchant_id'] = submission['id'].astype(str).str[6:]
submission['month_id'] = submission['id'].astype(str).str[:6]
submission.drop(columns=['net_payment_count'],inplace=True)

train.head()
test.head()


,merchant_id,month_id,merchant_source_name,settlement_period,working_type,mcc_id,merchant_segment,net_payment_count
0,merchant_43992,202307,Merchant Source - 3,Settlement Period - 3,Working Type - 2,mcc_197,Segment - 2,15106
1,merchant_43992,202301,Merchant Source - 3,Settlement Period - 3,Working Type - 2,mcc_197,Segment - 2,16918
2,merchant_43992,202305,Merchant Source - 3,Settlement Period - 3,Working Type - 2,mcc_197,Segment - 2,13452
3,merchant_43992,202308,Merchant Source - 3,Settlement Period - 3,Working Type - 2,mcc_197,Segment - 2,16787
4,merchant_43992,202302,Merchant Source - 3,Settlement Period - 3,Working Type - 2,mcc_197,Segment - 2,12428


,merchant_id,month_id,merchant_source_name,settlement_period,working_type,mcc_id,merchant_segment
0,merchant_36004,202311,Merchant Source - 1,Settlement Period - 1,Working Type - 2,mcc_144,Segment - 4
1,merchant_36004,202312,Merchant Source - 1,Settlement Period - 1,Working Type - 2,mcc_144,Segment - 4
2,merchant_36004,202310,Merchant Source - 1,Settlement Period - 1,Working Type - 2,mcc_144,Segment - 4
3,merchant_23099,202311,Merchant Source - 1,Settlement Period - 1,Working Type - 5,mcc_3,Segment - 4
4,merchant_23099,202312,Merchant Source - 1,Settlement Period - 1,Working Type - 5,mcc_3,Segment - 4


In [58]:
train=custom_df(train)
#train=train.sort_values(['merchant_id','date'])
test=custom_df(test)
train['month']=train.date.dt.month
train['year']=train.date.dt.year
test['month']=test.date.dt.month
test['year']=test.date.dt.year
train.head()
test.head()


,merchant_id,mcc_id,net_payment_count,date,merchant_source_name_id,settlement_period_id,working_type_id,merchant_segment_id,month,year
0,43992,197,15106,2023-07-01,3,3,2,2,7,2023
1,43992,197,16918,2023-01-01,3,3,2,2,1,2023
2,43992,197,13452,2023-05-01,3,3,2,2,5,2023
3,43992,197,16787,2023-08-01,3,3,2,2,8,2023
4,43992,197,12428,2023-02-01,3,3,2,2,2,2023


,merchant_id,mcc_id,date,merchant_source_name_id,settlement_period_id,working_type_id,merchant_segment_id,month,year
0,36004,144,2023-11-01,1,1,2,4,11,2023
1,36004,144,2023-12-01,1,1,2,4,12,2023
2,36004,144,2023-10-01,1,1,2,4,10,2023
3,23099,3,2023-11-01,1,1,5,4,11,2023
4,23099,3,2023-12-01,1,1,5,4,12,2023


## train

In [59]:
# train.info()
# print('-------')
# print('train shape:',train.shape)
# print('-------')
# print('train max date:',train.date.max())
# print('train min date:',train.date.min())
# print('-------')
# test.info()
# print('-------')
# print('test shape:',train.shape)
# print('-------')
# print('test max date:',test.date.max())
# print('test min date:',test.date.min())

* train verisi içerisindeki unique merchant_id sayısı(26060) ile test verisindeki eşit.Test verisinin shape i son üç ay göze alınarak oluşturulduğu için 3*26060=78180
+ veride   `merchant_source_name_id`,`settlement_period_id`,`working_type_id`,`merchant_segment_id` alanları `merchant_id` ile birebir kodluyor.


```python
train.groupby('merchant_id').agg({'merchant_source_name_id':'nunique','settlement_period_id':'nunique','working_type_id':'nunique','merchant_segment_id':'nunique'}).sum()
merchant_source_name_id    26060
settlement_period_id       26060
working_type_id            26060
merchant_segment_id        26060
dtype: int64
```

**Veri Setleri:**

- `merchant_id`: Maskelenmiş iş yeri ID'si
- `month_id`: İşlemin yapıldığı ay (YYYYMM formatında)
- `merchant_source`: İş yerinin iyzico’ya katıldığı kaynak
- `settlement_period`: İş yerinin hak edişini alış sıklığı
- `working_type`: İş yerinin tipini gösterir
- `mcc_id`: İş yerinin satış yaptığı kategori bilgisini gösterir
- `merchant_segment`: İş yerinin iyzico içerisinde bulunduğu segmenti gösterir
- `net_payment_count`: İş yerinin ilgili ay içerisinde geçirdiği net (ödeme - iptal - iade) işlem sayısıdır

**Submission Dosyası:**

- `id`: Tahminlenmesi beklenen ay ve maskelenmiş iş yeri ID'si (Örn: 202312merchant_36004)
- `net_payment_count`: İş yerinin ilgili ayda gerçekleştirdiği işlem sayısı


In [60]:
def train_col_summary_statistics(df, col1, col2):
    summary_stats = df.groupby(col1).agg(
        **{
            'NPC' + '_sum'+ '_by_' + col1 : (col2, 'sum'),
           'NPC' + '_mean'+ '_by_' + col1 : (col2, 'mean'),
            'NPC' + '_median'+ '_by_' + col1 : (col2, 'median'),
#             col2  + '_std'+ '_by_' + col1: (col2, 'std'),
            'NPC' + '_min'+ '_by_' + col1 : (col2, 'min'),
            'NPC' + '_max'+ '_by_' + col1 : (col2, 'max'),
            'NPC' + '_count'+ '_by_' + col1 : (col2, 'count')
        }
    )
    
    return summary_stats
train_col_summary_statistics(train,'merchant_source_name_id','net_payment_count').style.background_gradient(cmap='inferno', low=1, high=4)
train_col_summary_statistics(train,'settlement_period_id','net_payment_count').style.background_gradient(cmap='inferno', low=1, high=4)
train_col_summary_statistics(train,'working_type_id','net_payment_count').style.background_gradient(cmap='inferno', low=1, high=4)
train_col_summary_statistics(train,'merchant_segment_id','net_payment_count').style.background_gradient(cmap='inferno', low=1, high=4)
train_col_summary_statistics(train,'mcc_id','net_payment_count').head().style.background_gradient(cmap='inferno', low=1, high=4)
train_col_summary_statistics(train,'date','net_payment_count').head().style.background_gradient(cmap='inferno', low=1, high=4)


,NPC_sum_by_merchant_source_name_id,NPC_mean_by_merchant_source_name_id,NPC_median_by_merchant_source_name_id,NPC_min_by_merchant_source_name_id,NPC_max_by_merchant_source_name_id,NPC_count_by_merchant_source_name_id
merchant_source_name_id,,,,,,
1,36341190,220.254976,5.000000,-830,904587,164996
2,4833090,42.945149,6.000000,-112,78033,112541
3,79761507,5862.661301,106.000000,-1433,1160429,13605


,NPC_sum_by_settlement_period_id,NPC_mean_by_settlement_period_id,NPC_median_by_settlement_period_id,NPC_min_by_settlement_period_id,NPC_max_by_settlement_period_id,NPC_count_by_settlement_period_id
settlement_period_id,,,,,,
1,18642778,69.328234,5.000000,-1433,195879,268906
2,32734290,19449.964349,367.000000,-12,904587,1683
3,69558719,3384.358439,27.000000,-564,1160429,20553


,NPC_sum_by_working_type_id,NPC_mean_by_working_type_id,NPC_median_by_working_type_id,NPC_min_by_working_type_id,NPC_max_by_working_type_id,NPC_count_by_working_type_id
working_type_id,,,,,,
1,791,12.359375,4.500000,3,81,64
2,55890425,1394.784882,9.000000,-1433,1160429,40071
3,30966,5.950423,4.000000,-5,510,5204
4,4097,16.066667,5.000000,3,351,255
5,61566548,527.142449,6.000000,-139,904587,116793
6,3442960,26.740398,5.000000,-22,27664,128755


,NPC_sum_by_merchant_segment_id,NPC_mean_by_merchant_segment_id,NPC_median_by_merchant_segment_id,NPC_min_by_merchant_segment_id,NPC_max_by_merchant_segment_id,NPC_count_by_merchant_segment_id
merchant_segment_id,,,,,,
1,169588,495.871345,154.000000,3,7461,342
2,93196429,13434.687761,783.000000,-1433,1160429,6937
3,10616255,2899.031950,357.000000,-112,370127,3662
4,16953515,60.504834,5.000000,-830,87728,280201


,NPC_sum_by_mcc_id,NPC_mean_by_mcc_id,NPC_median_by_mcc_id,NPC_min_by_mcc_id,NPC_max_by_mcc_id,NPC_count_by_mcc_id
mcc_id,,,,,,
1,3082435,591.070949,7.000000,-107,122148,5215
2,3832,6.929476,4.000000,2,61,553
3,2830,12.358079,7.000000,3,59,229
4,678,6.582524,5.000000,3,33,103
5,121549,19.731981,6.000000,0,1025,6160


,NPC_sum_by_date,NPC_mean_by_date,NPC_median_by_date,NPC_min_by_date,NPC_max_by_date,NPC_count_by_date
date,,,,,,
2020-01-01 00:00:00,775289,206.798880,5.000000,-65,100155,3749
2020-02-01 00:00:00,759571,196.423843,5.000000,-19,130080,3867
2020-03-01 00:00:00,784726,198.614528,5.000000,-10,90452,3951
2020-04-01 00:00:00,906968,220.137864,6.000000,1,122178,4120
2020-05-01 00:00:00,1234374,262.242192,6.000000,-11,295715,4707


In [61]:
train.describe().style.background_gradient(cmap='inferno', low=1, high=4)

,merchant_id,mcc_id,net_payment_count,date,merchant_source_name_id,settlement_period_id,working_type_id,merchant_segment_id,month,year
count,291142.000000,291142.000000,291142.000000,291142,291142.000000,291142.000000,291142.000000,291142.000000,291142.000000,291142.000000
mean,33969.544095,82.611317,415.384201,2021-12-22 08:43:56.338282496,1.480010,1.146970,4.991836,3.936244,6.305875,2021.533750
min,1.000000,1.000000,-1433.000000,2020-01-01 00:00:00,1.000000,1.000000,1.000000,1.000000,1.000000,2020.000000
25%,16916.000000,40.000000,4.000000,2021-03-01 00:00:00,1.000000,1.000000,5.000000,4.000000,3.000000,2021.000000
50%,34152.000000,74.000000,6.000000,2022-01-01 00:00:00,1.000000,1.000000,5.000000,4.000000,6.000000,2022.000000
75%,50911.500000,130.000000,14.000000,2022-11-01 00:00:00,2.000000,1.000000,6.000000,4.000000,9.000000,2022.000000
max,67816.000000,197.000000,1160429.000000,2023-09-01 00:00:00,3.000000,3.000000,6.000000,4.000000,12.000000,2023.000000
std,19575.793939,51.638031,10820.674338,nan,0.585714,0.516294,1.325434,0.338221,3.355080,1.048089


In [62]:
interval3=(train.date.max() - pd.DateOffset(months=3),train.date.max())
interval6=(train.date.max() - pd.DateOffset(months=6),train.date.max())
interval9=(train.date.max() - pd.DateOffset(months=9),train.date.max())

dum=train.groupby('merchant_id').agg(son3ay_aktiflik=('date',lambda x: len(list(filter(lambda d: interval3[0] <= d <= interval3[1], x)))),
                                     son6ay_aktiflik=('date',lambda x: len(list(filter(lambda d: interval6[0] <= d <= interval6[1], x)))),
                                     son9ay_aktiflik=('date',lambda x: len(list(filter(lambda d: interval9[0] <= d <= interval9[1], x))))
                                     ).fillna(0)


filtered_data3 = train[(train.date >= interval3[0]) & (train.date <= interval3[1])]
filtered_data6= train[(train.date >= interval6[0]) & (train.date <= interval6[1])]
filtered_data9 = train[(train.date >= interval9[0]) & (train.date <= interval9[1])]
dum3Mean=filtered_data3.groupby('merchant_id').agg(son3ay_mean=('net_payment_count','mean'))
dum6Mean=filtered_data6.groupby('merchant_id').agg(son6ay_mean=('net_payment_count','mean'))
dum9Mean=filtered_data9.groupby('merchant_id').agg(son9ay_mean=('net_payment_count','mean'))

##Train
train=train.merge(dum3Mean,on='merchant_id',how='left').merge(dum6Mean,on='merchant_id',how='left').merge(dum9Mean,on='merchant_id',how='left').fillna(0)
train=train.merge(dum,on='merchant_id',how='left')
##Test
test=test.merge(dum3Mean,on='merchant_id',how='left').merge(dum6Mean,on='merchant_id',how='left').merge(dum9Mean,on='merchant_id',how='left').fillna(0)
test=test.merge(dum,on='merchant_id',how='left')
train.head()
test.head()


,merchant_id,mcc_id,net_payment_count,date,merchant_source_name_id,settlement_period_id,working_type_id,merchant_segment_id,month,year,son3ay_mean,son6ay_mean,son9ay_mean,son3ay_aktiflik,son6ay_aktiflik,son9ay_aktiflik
0,43992,197,15106,2023-07-01,3,3,2,2,7,2023,15611.0,15279.857143,15254.8,4,7,10
1,43992,197,16918,2023-01-01,3,3,2,2,1,2023,15611.0,15279.857143,15254.8,4,7,10
2,43992,197,13452,2023-05-01,3,3,2,2,5,2023,15611.0,15279.857143,15254.8,4,7,10
3,43992,197,16787,2023-08-01,3,3,2,2,8,2023,15611.0,15279.857143,15254.8,4,7,10
4,43992,197,12428,2023-02-01,3,3,2,2,2,2023,15611.0,15279.857143,15254.8,4,7,10


,merchant_id,mcc_id,date,merchant_source_name_id,settlement_period_id,working_type_id,merchant_segment_id,month,year,son3ay_mean,son6ay_mean,son9ay_mean,son3ay_aktiflik,son6ay_aktiflik,son9ay_aktiflik
0,36004,144,2023-11-01,1,1,2,4,11,2023,0.0,0.0,0.0,0,0,0
1,36004,144,2023-12-01,1,1,2,4,12,2023,0.0,0.0,0.0,0,0,0
2,36004,144,2023-10-01,1,1,2,4,10,2023,0.0,0.0,0.0,0,0,0
3,23099,3,2023-11-01,1,1,5,4,11,2023,3.0,3.0,3.0,1,2,2
4,23099,3,2023-12-01,1,1,5,4,12,2023,3.0,3.0,3.0,1,2,2


**Veri Setleri:**

- `merchant_id`: Maskelenmiş iş yeri ID'si
- `month_id`: İşlemin yapıldığı ay (YYYYMM formatında)
- `merchant_source`: İş yerinin iyzico’ya katıldığı kaynak
- `settlement_period`: İş yerinin hak edişini alış sıklığı
- `working_type`: İş yerinin tipini gösterir
- `mcc_id`: İş yerinin satış yaptığı kategori bilgisini gösterir
- `merchant_segment`: İş yerinin iyzico içerisinde bulunduğu segmenti gösterir
- `net_payment_count`: İş yerinin ilgili ay içerisinde geçirdiği net (ödeme - iptal - iade) işlem sayısıdır

**Submission Dosyası:**

- `id`: Tahminlenmesi beklenen ay ve maskelenmiş iş yeri ID'si (Örn: 202312merchant_36004)
- `net_payment_count`: İş yerinin ilgili ayda gerçekleştirdiği işlem sayısı


In [63]:
dum=train.groupby(['month','working_type_id']).agg(month_working_type_id=('net_payment_count','median'))
train=train.merge(dum,on=['month','working_type_id'],how='left')
test=test.merge(dum,on=['month','working_type_id'],how='left')
dum2=train.groupby(['month','mcc_id']).agg(month_mcc_id=('net_payment_count','median'))
train=train.merge(dum2,on=['month','mcc_id'],how='left')
test=test.merge(dum2,on=['month','mcc_id'],how='left')

In [65]:
train.head()

,merchant_id,mcc_id,net_payment_count,date,merchant_source_name_id,settlement_period_id,working_type_id,merchant_segment_id,month,year,son3ay_mean,son6ay_mean,son9ay_mean,son3ay_aktiflik,son6ay_aktiflik,son9ay_aktiflik,month_working_type_id,month_mcc_id
0,43992,197,15106,2023-07-01,3,3,2,2,7,2023,15611.0,15279.857143,15254.8,4,7,10,9.0,5922.5
1,43992,197,16918,2023-01-01,3,3,2,2,1,2023,15611.0,15279.857143,15254.8,4,7,10,9.0,6074.5
2,43992,197,13452,2023-05-01,3,3,2,2,5,2023,15611.0,15279.857143,15254.8,4,7,10,9.0,6395.5
3,43992,197,16787,2023-08-01,3,3,2,2,8,2023,15611.0,15279.857143,15254.8,4,7,10,9.0,7957.0
4,43992,197,12428,2023-02-01,3,3,2,2,2,2023,15611.0,15279.857143,15254.8,4,7,10,8.0,5670.5


+ Her merchant_id unique  mcc_id'ye sahip yani bir şirket 2 ayrı kategoride satış yapmıyor ,bazı şirketlerin kategorileri aynı.

```python
train.groupby('merchant_id').agg({'mcc_id':'nunique'}).max()
mcc_id    1
dtype: int64
```


+ farklı bir kullanım örneği
```python
median_value = train[train['merchant_id'] == 5253]['net_payment_count'].median() * 0.02
filtered_data = train.query('net_payment_count < @median_value') 
```

In [66]:
dum=train.groupby('merchant_id').agg(date_nunqiue_by_merchant_id=('date','nunique'),
#                                 NPC__median_by_merchant_id=('net_payment_count','median'),
                                NPC__mean_by_merchant_id=('net_payment_count','mean'),
                                 NPC__sum_by_merchant_id=('net_payment_count','sum'),
                          #       NPC__min_by_merchant_id=('net_payment_count','min'),
                             #    NPC__max_by_merchant_id=('net_payment_count','max'),
                                 date_range_day=('date', lambda x: (x.max() - x.min()).days)

                                )

train=train.merge(dum,on='merchant_id',how='left')
test=test.merge(dum,on='merchant_id',how='left')

In [67]:
# dum=train.groupby('mcc_id').agg(NPC_sum_by_mcc_id=('net_payment_count','sum'),
#                             NPC_mean_by_mcc_id=('net_payment_count','mean'),
#                             NPC_median_by_mcc_id=('net_payment_count','median'),
#                         #    NPC_max_by_mcc_id=('net_payment_count','max'),
#                         #    NPC_min_by_mcc_id=('net_payment_count','min')
#                                )#.style.background_gradient(cmap='inferno', low=1, high=4)
# train=train.merge(dum,on='mcc_id',how='left')
# test=test.merge(dum,on='mcc_id',how='left')

In [68]:

# 'merchant_id' grubuna göre 'net_payment_count' sütununun median değerlerini hesaplayalım
median_by_merchant_id = train.groupby('merchant_id')['net_payment_count'].median()
# Özel işlem için kullanılacak fonksiyon
def custom_function(x):
    return np.sum([1 if val < median_by_merchant_id[x.name] * 0.02 else 0 for val in x])/len([1 if val < median_by_merchant_id[x.name] * 0.02 else 0 for val in x])
# 'net_payment_count' sütunu üzerinde custom_function'ı uygulayalım
result = train.groupby('merchant_id')['net_payment_count'].apply(custom_function).reset_index(name='normal_olmayan_NPC_month_oranı')
train=train.merge(result,on='merchant_id',how='left')
test=test.merge(result,on='merchant_id',how='left')



In [69]:
# data=train.groupby(['mcc_id','date']).agg({'net_payment_count':'sum','net_payment_count':'mean'}).reset_index()
# sns.set(rc={'figure.figsize':(25,10)})
# sns.lineplot(x=data.date, y="net_payment_count", hue=data.mcc_id,data=data).set_title('tüketim')


In [98]:
target=train.net_payment_count
df_train=train.drop('net_payment_count',axis=1)
df_test=test.copy()

df_train=df_train.drop(['date'],axis=1)
df_test=df_test.drop(['date'],axis=1)

In [99]:
df_train

,merchant_id,mcc_id,merchant_source_name_id,settlement_period_id,working_type_id,merchant_segment_id,month,year,son3ay_mean,son6ay_mean,son9ay_mean,son3ay_aktiflik,son6ay_aktiflik,son9ay_aktiflik,month_working_type_id,month_mcc_id,date_nunqiue_by_merchant_id,NPC__mean_by_merchant_id,NPC__sum_by_merchant_id,date_range_day,normal_olmayan_NPC_month_oranı
0,43992,197,3,3,2,2,7,2023,15611.0,15279.857143,15254.8,4,7,10,9.0,5922.5,45,8509.688889,382936,1339,0.0
1,43992,197,3,3,2,2,1,2023,15611.0,15279.857143,15254.8,4,7,10,9.0,6074.5,45,8509.688889,382936,1339,0.0
2,43992,197,3,3,2,2,5,2023,15611.0,15279.857143,15254.8,4,7,10,9.0,6395.5,45,8509.688889,382936,1339,0.0
3,43992,197,3,3,2,2,8,2023,15611.0,15279.857143,15254.8,4,7,10,9.0,7957.0,45,8509.688889,382936,1339,0.0
4,43992,197,3,3,2,2,2,2023,15611.0,15279.857143,15254.8,4,7,10,8.0,5670.5,45,8509.688889,382936,1339,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291137,67083,169,1,1,6,4,9,2023,4.0,4.000000,4.0,1,1,1,5.0,6.0,1,4.000000,4,0,0.0
291138,4263,73,2,1,6,4,9,2023,4.0,4.000000,4.0,1,1,1,5.0,5.0,1,4.000000,4,0,0.0
291139,11346,169,1,1,6,4,9,2023,3.0,3.000000,3.0,1,1,1,5.0,6.0,1,3.000000,3,0,0.0
291140,21397,168,1,1,6,4,9,2023,3.0,3.000000,3.0,1,1,1,5.0,5.0,1,3.000000,3,0,0.0


In [100]:
df_test

,merchant_id,mcc_id,merchant_source_name_id,settlement_period_id,working_type_id,merchant_segment_id,month,year,son3ay_mean,son6ay_mean,son9ay_mean,son3ay_aktiflik,son6ay_aktiflik,son9ay_aktiflik,month_working_type_id,month_mcc_id,date_nunqiue_by_merchant_id,NPC__mean_by_merchant_id,NPC__sum_by_merchant_id,date_range_day,normal_olmayan_NPC_month_oranı
0,36004,144,1,1,2,4,11,2023,0.0,0.0,0.0,0,0,0,9.0,7.0,4,5.750000,23,214,0.0
1,36004,144,1,1,2,4,12,2023,0.0,0.0,0.0,0,0,0,9.0,6.0,4,5.750000,23,214,0.0
2,36004,144,1,1,2,4,10,2023,0.0,0.0,0.0,0,0,0,9.0,6.0,4,5.750000,23,214,0.0
3,23099,3,1,1,5,4,11,2023,3.0,3.0,3.0,1,2,2,6.0,7.0,2,3.000000,6,61,0.0
4,23099,3,1,1,5,4,12,2023,3.0,3.0,3.0,1,2,2,6.0,9.5,2,3.000000,6,61,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78175,35969,110,1,1,6,4,12,2023,0.0,0.0,0.0,0,0,0,5.0,6.0,1,3.000000,3,0,0.0
78176,35969,110,1,1,6,4,10,2023,0.0,0.0,0.0,0,0,0,5.0,5.0,1,3.000000,3,0,0.0
78177,8429,42,1,1,5,4,11,2023,0.0,0.0,0.0,0,0,0,6.0,6.0,19,110.947368,2108,670,0.0
78178,8429,42,1,1,5,4,12,2023,0.0,0.0,0.0,0,0,0,6.0,6.0,19,110.947368,2108,670,0.0


In [113]:
def objective(trial,data=df_train,target=target):
    
        param = {
            'base_score':0.5,
            'objective':'reg:squarederror',
          #  'booster':'gbtree', 
            'lambda': trial.suggest_float('lambda', 1e-3, 10),
            'alpha': trial.suggest_float('alpha', 1e-3, 10),
            'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.7,0.8,0.9,1.0]),
            'subsample': trial.suggest_categorical('subsample', [0.5,0.6,0.7,0.8,1.0]),
            'learning_rate': trial.suggest_categorical('learning_rate', [0.08,0.009,0.01,0.012,0.014,0.016,0.018,0.001,0.1,0.5]),
            'n_estimators':500,
            'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11]),
            'random_state': trial.suggest_categorical('random_state', [24, 48,2020]),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 500),
            'tree_method':'gpu_hist',
#             'eval_metric': "mape",
      #      'enable_categorical':True,
        #    'early_stopping_rounds':250
        } 

        best_iteration_list=[]
        score_list=[]
        model_list=[]
        fea_importance_list=[]
        
        kf = KFold(n_splits=5 ,shuffle=False)
        kf_splits = list(kf.split(data,target))  
        for split_train, split_val in kf_splits:
            x_train, y_train = data.iloc[split_train], target.iloc[split_train]
            x_val, y_val = data.iloc[split_val], target.iloc[split_val]
        
   
        model = xgb.XGBRegressor(**param)  
        model.fit(x_train,y_train,eval_set=[(x_train, y_train), (x_val, y_val)],verbose=0)
       # best_iteration=model.best_iteration
        #best_ntree_limit=model.best_ntree_limit
        val_pred=model.predict(x_val)
        model_list.append(model)
        score=mean_absolute_error(y_val, val_pred)
        score_list.append(score)
        fea_importance_list.append(model.feature_importances_)


        trial.set_user_attr('model',model_list)
       # trial.set_user_attr('best_ntree_limit',best_ntree_limit)
      #  trial.set_user_attr('best_iteration',best_iteration)
        trial.set_user_attr("feature_importance", np.mean(fea_importance_list,axis=0))
        trial.set_user_attr("score_list", score_list)

        scores=np.mean(score)
        return scores



study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=15)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2024-01-27 18:32:27,527] A new study created in memory with name: no-name-0dcba93a-ca9c-4717-add3-87ac84f63391
[I 2024-01-27 18:32:29,936] Trial 0 finished with value: 1042.4963027760186 and parameters: {'lambda': 7.35184754389607, 'alpha': 9.359880285302255, 'colsample_bytree': 0.9, 'subsample': 1.0, 'learning_rate': 0.5, 'max_depth': 5, 'random_state': 48, 'min_child_weight': 313}. Best is trial 0 with value: 1042.4963027760186.
[I 2024-01-27 18:32:32,638] Trial 1 finished with value: 423.8999780219642 and parameters: {'lambda': 4.189205316829706, 'alpha': 1.4968608559958214, 'colsample_bytree': 0.8, 'subsample': 0.7, 'learning_rate': 0.014, 'max_depth': 7, 'random_state': 24, 'min_child_weight': 494}. Best is trial 1 with value: 423.8999780219642.
[I 2024-01-27 18:32:34,978] Trial 2 finished with value: 411.5594124615094 and parameters: {'lambda': 8.408515657086722, 'alpha': 9.712140228166207, 'colsample_bytree': 1.0, 'subsample': 0.7, 'learning_rate': 0.016, 'max_depth': 5, 'ran

Number of finished trials: 15
Best trial: {'lambda': 0.1736107123144306, 'alpha': 7.285812975053502, 'colsample_bytree': 0.7, 'subsample': 0.6, 'learning_rate': 0.001, 'max_depth': 11, 'random_state': 2020, 'min_child_weight': 26}


In [114]:
param={
'lambda': 0.1736107123144306, 'alpha': 7.285812975053502, 'colsample_bytree': 0.7, 'subsample': 0.6, 'learning_rate': 0.001, 'max_depth': 11, 'random_state': 2020, 'min_child_weight': 26,
    'n_estimators':500,
    'base_score':0.5,
            'objective':'reg:squarederror',
#'enable_categorical':True,
} 

all_models=[]
for i in [1200, 54,55]:
    param["random_state"] = i
    model = xgb.XGBRegressor(**param)  
    model.fit(df_train,target)
    all_models.append(model)

preds = [model.predict(df_test)  for model in all_models]
mean_preds = np.mean(preds, axis=0)
mean_preds[:150]

XGBRegressor(alpha=7.285812975053502, base_score=0.5, booster=None,
             callbacks=None, colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.7, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, lambda=0.1736107123144306,
             learning_rate=0.001, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=11,
             max_leaves=None, min_child_weight=26, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=500,
             n_jobs=None, ...)

XGBRegressor(alpha=7.285812975053502, base_score=0.5, booster=None,
             callbacks=None, colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.7, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, lambda=0.1736107123144306,
             learning_rate=0.001, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=11,
             max_leaves=None, min_child_weight=26, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=500,
             n_jobs=None, ...)

XGBRegressor(alpha=7.285812975053502, base_score=0.5, booster=None,
             callbacks=None, colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.7, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, lambda=0.1736107123144306,
             learning_rate=0.001, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=11,
             max_leaves=None, min_child_weight=26, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=500,
             n_jobs=None, ...)

array([ 2.6926594,  2.6926594,  2.6926594,  1.9596318,  1.9596318,
        1.9596318,  2.0716794,  2.0716794,  2.0716794,  2.1050804,
        2.1050804,  2.1050804,  1.9602824,  1.9602824,  1.9602824,
        2.0599868,  2.0599868,  2.0599868,  2.0331821,  2.0331821,
        2.0331821,  1.9602824,  1.9602824,  1.9602824,  2.5989335,
        2.5989335,  2.5989335,  3.7796357,  3.7802398,  3.7796357,
        2.737141 ,  2.737141 ,  2.737141 ,  2.2594867,  2.2594867,
        2.2594867,  1.9913639,  1.9913639,  1.9913639,  7.9475217,
        7.9996333,  7.9456844,  2.1448052,  2.1448052,  2.1448052,
        2.1433156,  2.1433156,  2.1433156,  1.9704496,  1.9704496,
        1.9704496,  2.0331821,  2.0331821,  2.0331821,  3.8228157,
        3.8228157,  3.8228157,  1.9610382,  1.9610382,  1.9610382,
        2.2553985,  2.2553985,  2.2553985,  2.1649396,  2.1649396,
        2.1649396,  2.0356016,  2.0356016,  2.0356016,  3.455403 ,
        3.4567497,  3.455403 ,  2.0331821,  2.0331821,  2.0331

In [94]:
sub=pd.read_csv('/kaggle/input/iyzico-datathon/sample_submission.csv')
sub.net_payment_count=mean_preds
sub=sub.set_index('id')
sub.to_csv('result1.csv')